# Fitting a Model With Data Augmentation

In [1]:
import os
print(os.listdir("../input/sohan-dataset-new/augmented asl 3/Augmented ASL 3"))
    
    

['F', 'V', 'Q', 'I', 'N', 'W', 'D', 'C', 'A', 'E', 'X', 'Y', 'K', 'O', 'L', 'P', 'del', 'R', 'B', 'G', 'Z', 'M', 'H', 'U', 'nothing', 'S', 'T', 'J', 'space']


In [2]:
from tensorflow.python.keras.applications.resnet50 import preprocess_input
from tensorflow.python.keras.preprocessing.image import ImageDataGenerator

image_size = 64

data_generator_with_aug = ImageDataGenerator(
                                    samplewise_center=True, 
                                    samplewise_std_normalization=True,
                                   width_shift_range = 0.1,
                                   height_shift_range = 0.1,
                                   validation_split = 0.1, rotation_range = 10,
                                   zoom_range = 0.1)


train_generator = data_generator_with_aug.flow_from_directory(
        '../input/asl-alphabet/asl_alphabet_train/asl_alphabet_train',
        target_size=(image_size, image_size),
        batch_size=64,
        class_mode='categorical', subset = 'training')

validation_generator = data_generator_with_aug.flow_from_directory(
        '../input/asl-alphabet/asl_alphabet_train/asl_alphabet_train',
        target_size=(image_size, image_size),
        class_mode='categorical', subset = 'validation')



Found 78300 images belonging to 29 classes.
Found 8700 images belonging to 29 classes.


In [3]:
from tensorflow.python.keras.applications import ResNet50
from tensorflow.python.keras.models import Sequential
from tensorflow.python.keras.layers import Dense, Flatten, GlobalAveragePooling2D

num_classes = 29
resnet_weights_path = '../input/resnet50/resnet50_weights_tf_dim_ordering_tf_kernels_notop.h5'

my_new_model = Sequential()
my_new_model.add(ResNet50(include_top=False, pooling='avg', weights=resnet_weights_path))
my_new_model.add(Dense(num_classes, activation='softmax'))

# Say not to train first layer (ResNet) model. It is already trained
my_new_model.layers[0].trainable = False

my_new_model.compile(optimizer='sgd', loss='categorical_crossentropy', metrics=['accuracy'])

In [4]:
my_new_model.fit_generator(
        train_generator,
        steps_per_epoch=10,
        epochs=6,
        validation_data=validation_generator, validation_steps = 20)

Epoch 1/6
10/10 [==============================] - 11s 1s/step - loss: 4.0820 - acc: 0.0391 - val_loss: 3.6270 - val_acc: 0.0328
Epoch 2/6
10/10 [==============================] - 4s 364ms/step - loss: 3.8700 - acc: 0.0687 - val_loss: 3.5686 - val_acc: 0.0406
Epoch 3/6
10/10 [==============================] - 4s 365ms/step - loss: 3.6936 - acc: 0.0594 - val_loss: 3.5440 - val_acc: 0.0516
Epoch 4/6
10/10 [==============================] - 4s 364ms/step - loss: 3.4364 - acc: 0.0953 - val_loss: 3.5619 - val_acc: 0.0359
Epoch 5/6
10/10 [==============================] - 4s 364ms/step - loss: 3.3424 - acc: 0.1187 - val_loss: 3.6499 - val_acc: 0.0312
Epoch 6/6
10/10 [==============================] - 4s 363ms/step - loss: 3.2480 - acc: 0.1484 - val_loss: 3.7465 - val_acc: 0.0234


In [5]:
my_new_model.save("trial.hdf5")